In [ ]:
# Imports
import numpy as np
import cv2
import math

# Open Camera
capture = cv2.VideoCapture(0)

while capture.isOpened():
    # Capture frames from the camera
    ret, frame = capture.read()

    if not ret:
        print("Failed to grab frame")
        break

    # Draw rectangle on the frame
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)
    crop_image = frame[100:300, 100:300]

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(crop_image, (3, 3), 0)

    # Change color-space from BGR to HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # Create a binary image where skin color is white and rest is black
    mask2 = cv2.inRange(hsv, np.array([0, 20, 50), np.array([20, 255, 255]))

    # Kernel for morphological transformations    
    kernel = np.ones((7, 7), np.uint8)

    # Apply morphological transformations to filter out background noise
    dilation = cv2.dilate(mask2, kernel, iterations=1)
    erosion = cv2.erode(dilation, kernel, iterations=1)

    # Apply Gaussian Blur and Threshold
    filtered = cv2.GaussianBlur(erosion, (3, 3), 0)
    ret, thresh = cv2.threshold(filtered, 127, 255, 0)

    # Show threshold image
    cv2.imshow("Thresholded", thresh)

    # Find contours (fixed for OpenCV 4.x)
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        # Find contour with maximum area
        contour = max(contours, key=lambda x: cv2.contourArea(x))

        # Create bounding rectangle around the contour
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image, (x, y), (x+w, y+h), (0, 0, 255), 0)

        # Find convex hull
        hull = cv2.convexHull(contour)

        # Draw contour
        drawing = np.zeros(crop_image.shape, np.uint8)
        cv2.drawContours(drawing, [contour], -1, (0, 255, 0), 0)
        cv2.drawContours(drawing, [hull], -1, (0, 0, 255), 0)

        # Find convexity defects
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hull)

        if defects is not None:
            count_defects = 0
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])

                # Find length of sides of triangle
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)

                # Apply cosine rule here
                angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / math.pi

                # Ignore angles > 90 and highlight rest with red dots
                if angle <= 90:
                    count_defects += 1
                    cv2.circle(crop_image, far, 5, [0, 0, 255], -1)

                cv2.line(crop_image, start, end, [0, 255, 0], 2)

            # Gesture Recognition Based on Defects
            if count_defects == 0:
                cv2.putText(frame, "Fist", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)
            elif count_defects == 1:
                cv2.putText(frame, "tumbsup", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)
            elif count_defects == 2:
                cv2.putText(frame, "Peace Sign", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)
            elif count_defects == 3:
                cv2.putText(frame, "Okay", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)
            elif count_defects == 4:
                cv2.putText(frame, "HI", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)
            else:
                pass

    except Exception as e:
        print("Error:", e)
        pass

    # Show the required images
    cv2.imshow("Gesture", frame)
    if 'drawing' in locals():
        all_image = np.hstack((drawing, crop_image))
        cv2.imshow('Contours', all_image)

    # Close the camera if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()
